<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#load-dataset" data-toc-modified-id="load-dataset-1">load dataset</a></span></li><li><span><a href="#train-model" data-toc-modified-id="train-model-2">train model</a></span></li></ul></div>

# load dataset

In [1]:
import os
from functools import partial
import wandb
from wandb.keras import WandbCallback
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.applications import EfficientNetB2

In [2]:
config = {
    "image_size": 260,
    "num_classes": 55,
    "batch_size": 16,
    "learning_rate": 0.0002,
    "metric": "accuracy",
    "epoch": 30
}

In [3]:
wandb.init(
    project="place_55_210614",
    config=config
          )

wandb: Currently logged in as: kec0130 (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.32 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [4]:
config = wandb.config

In [5]:
def _parse_function(tfrecord_serialized, image_size):
    features = {'image': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.int64)
               }
    parsed_features = tf.io.parse_single_example(tfrecord_serialized, features)

    image = tf.io.decode_raw(parsed_features['image'], tf.uint8)
    image = tf.reshape(image, [224, 224, 3])
    image = tf.image.resize(image, [image_size, image_size])

    label = tf.cast(parsed_features['label'], tf.int64)
    label = tf.one_hot(label, config.num_classes)

    return image, label

In [6]:
train_path = './storage/data/place_55_74604_train_shuffle.tfr'
train_dataset = tf.data.TFRecordDataset(train_path)

train_ds = train_dataset.map(
    partial(_parse_function, image_size=config.image_size), 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

train_ds = train_ds.shuffle(config.batch_size * 100)
train_ds = train_ds.batch(config.batch_size)
# train_ds = train_ds.repeat()
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [7]:
val_path = './storage/data/place_55_21516_val_shuffle.tfr'
val_dataset = tf.data.TFRecordDataset(val_path)

val_ds = val_dataset.map(
    partial(_parse_function, image_size=config.image_size), 
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

val_ds = val_ds.batch(config.batch_size)

# train model

In [8]:
data_augmentation = tf.keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(factor=0.1),
        preprocessing.RandomContrast(factor=0.1),
        preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1),
    ]
)

In [9]:
base_model = EfficientNetB2(
    input_shape=(config.image_size, config.image_size, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

In [10]:
base_model.summary()

Model: "efficientnetb2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 260, 260, 3) 0                                            
__________________________________________________________________________________________________
rescaling (Rescaling)           (None, 260, 260, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
normalization (Normalization)   (None, 260, 260, 3)  7           rescaling[0][0]                  
__________________________________________________________________________________________________
stem_conv_pad (ZeroPadding2D)   (None, 261, 261, 3)  0           normalization[0][0]              
_____________________________________________________________________________________

block5a_se_expand (Conv2D)      (None, 1, 1, 528)    12144       block5a_se_reduce[0][0]          
__________________________________________________________________________________________________
block5a_se_excite (Multiply)    (None, 17, 17, 528)  0           block5a_activation[0][0]         
                                                                 block5a_se_expand[0][0]          
__________________________________________________________________________________________________
block5a_project_conv (Conv2D)   (None, 17, 17, 120)  63360       block5a_se_excite[0][0]          
__________________________________________________________________________________________________
block5a_project_bn (BatchNormal (None, 17, 17, 120)  480         block5a_project_conv[0][0]       
__________________________________________________________________________________________________
block5b_expand_conv (Conv2D)    (None, 17, 17, 720)  86400       block5a_project_bn[0][0]         
__________

In [11]:
input_layer = tf.keras.layers.Input((config.image_size, config.image_size, 3))
model = data_augmentation(input_layer)
model = base_model(model)
# model = tf.keras.layers.Dense(256, activation='relu')(model)
# model = tf.keras.layers.BatchNormalization()(model)
model = tf.keras.layers.Dense(config.num_classes)(model)
model = tf.keras.Model(input_layer, model)

In [12]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 260, 260, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 260, 260, 3)       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 1408)              7768569   
_________________________________________________________________
dense (Dense)                (None, 55)                77495     
Total params: 7,846,064
Trainable params: 7,778,489
Non-trainable params: 67,575
_________________________________________________________________


In [13]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=[config.metric]
)

In [14]:
save_dir = './storage/models/'
if not os.path.exists(save_dir):
    os.mkdir(save_dir)

In [15]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
mc = tf.keras.callbacks.ModelCheckpoint(
    filepath=save_dir+'efn_b2_{epoch}-{val_loss:.2f}-{val_accuracy:.2f}.h5',
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

In [16]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', verbose=1, factor=0.2, patience=2)

In [17]:
wan = WandbCallback()

In [18]:
history = model.fit(
    train_ds,
#     steps_per_epoch=int(TRAIN_SIZE/BATCH_SIZE),
#     validation_steps=int(VAL_SIZE/BATCH_SIZE),
    epochs=config.epoch,
    validation_data=val_ds,
    callbacks=[reduce_lr, es, mc, wan]
)

Epoch 1/30
      6/Unknown - 193s 195ms/step - loss: 4.0062 - accuracy: 0.0215 WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0661s vs `on_train_batch_end` time: 0.1081s). Check your callbacks.
4663/4663 [==============================] - 1280s 233ms/step - loss: 1.2501 - accuracy: 0.6795 - val_loss: 0.4736 - val_accuracy: 0.8637

Epoch 00001: val_loss improved from inf to 0.47360, saving model to ./storage/models/efn_b2_1-0.47-0.86.h5
Epoch 2/30
4663/4663 [==============================] - 914s 196ms/step - loss: 0.5201 - accuracy: 0.8499 - val_loss: 0.5164 - val_accuracy: 0.8575

Epoch 00002: val_loss did not improve from 0.47360
Epoch 3/30
4663/4663 [==============================] - 916s 196ms/step - loss: 0.4104 - accuracy: 0.8779 - val_loss: 0.4413 - val_accuracy: 0.8824

Epoch 00003: val_loss improved from 0.47360 to 0.44126, saving model to ./storage/models/efn_b2_3-0.44-0.88.h5
Epoch 4/30
4663/4663 [===================